In [ ]:
#=============================================================
# 패키지 로딩
#============================================================
import os
import sys
import glob
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import h2o
import numpy as np
import itertools 
import math
import datetime

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.grid.grid_search import H2OGridSearch
from pandasql import sqldf

In [ ]:
#============================================================
# 분석 환경 셋팅
#============================================================
sys.stdout.flush() #Python 메모리에 생성된 모든 객체 삭제(초기화)

#============================================================
# 작업 디렉토리 경로 확인
#============================================================
currentPath=os.getcwd()
print('Current working dir : %s' % currentPath)

In [ ]:
#============================================================
# 기상 데이터 읽어오기 
#============================================================
ASOS = pd.read_csv(currentPath + "/input/ASOS_imput.csv", encoding='UTF-8') #loading weather data
BUOY_DP = pd.read_csv(currentPath + "/input/BUOY_DP_imput.csv", encoding='UTF-8') #loading weather data
HYCOM = pd.read_csv(currentPath + "/input/SEA_IVs_hycom_all.csv", encoding='UTF-8') #loading hycom data

ASOS = ASOS.rename(columns = {"WS_MAX":"WS_MAX_ASOS"})
BUOY_DP = BUOY_DP.rename(columns = {"WS_MAX":"WS_MAX_BD"})

#=============================================================
# 불러온 데이터 구조 확인하기
#=============================================================
ASOS
BUOY_DP
HYCOM

In [ ]:
#=============================================================
# 테이블 결합 및 확인 
#=============================================================
HYCOM['YYMMDD'] = pd.to_datetime(HYCOM['YYMMDD'], format='%Y-%m-%d')
HYCOM[['year', 'month']] = HYCOM[['year', 'month']].apply(pd.to_numeric)
HYCOM.dtypes

ASOS['YYMMDD'] = pd.to_datetime(ASOS['YYMMDD'], format='%Y-%m-%d')
ASOS[['year', 'month']] = ASOS[['year', 'month']].apply(pd.to_numeric)
ASOS.dtypes

BUOY_DP['YYMMDD'] = pd.to_datetime(BUOY_DP['YYMMDD'], format='%Y-%m-%d')
BUOY_DP[['year', 'month']] = BUOY_DP[['year', 'month']].apply(pd.to_numeric)
BUOY_DP.dtypes

DT = pd.merge(HYCOM, ASOS, how='left', on=['HAEGU_NUM', 'YYMMDD', 'year', 'month'])
DT = pd.merge(DT, BUOY_DP, how='left', on=['HAEGU_NUM', 'YYMMDD', 'year', 'month'])

DT.columns.values

In [ ]:
#============================================================
# 적조 데이터 읽어오기 
#============================================================
redtide = pd.read_csv(currentPath + "/input/redtide.csv", sep='\t', encoding='UTF-8') #loading redtide data

# column 수정
redtide.rename(columns={redtide.columns[0]:"YYMMDD"}, inplace = True)
redtide.rename(columns={redtide.columns[5]:"LAT_r"}, inplace = True)
redtide.rename(columns={redtide.columns[6]:"LON_r"}, inplace = True)

redtide['YYMMDD'] = pd.to_datetime(redtide['YYMMDD'], format='%Y%m%d')
redtide['month'] = pd.to_numeric(redtide['YYMMDD'].dt.month)
redtide['year'] = pd.to_numeric(redtide['YYMMDD'].dt.year)

#============================================================
# 불러온 데이터 구조 확인하기 
#============================================================
redtide.info()

In [ ]:
#=============================================================
# 해구 데이터 읽어오기
#=============================================================
HAEGU = pd.read_csv(currentPath + "/input/SEA_latlon.csv", sep='\t', encoding='UTF-8') #loading redtide data
HAEGU = HAEGU.sort_values(by=['HAEGU_NUM'])
HAEGU['row_h'] = HAEGU.index + 1

HAEGU.rename(columns={HAEGU.columns[1]:"LAT_h"}, inplace = True)
HAEGU.rename(columns={HAEGU.columns[2]:"LON_h"}, inplace = True)

#============================================================
# 불러온 데이터 구조 확인하기 
#============================================================
HAEGU.info()

In [ ]:
#=============================================================
# 테이블 결합 및 확인
#=============================================================
match=redtide[["LAT_r","LON_r"]].drop_duplicates()
match["row_r"]=match.index+1

def expand_grid(data_dict):
    rows = itertools.product(*data_dict.values())
    return pd.DataFrame.from_records(rows, columns=data_dict.keys())
tmp = expand_grid({'row_h' : HAEGU['row_h'], 'row_r' : match['row_r']})
np.shape(tmp)[0]# nrow(match)*nrow(HAEGU)= 668*1318

tmp = pd.merge(tmp, HAEGU, how='left', on=['row_h'])
tmp = pd.merge(tmp, match, how='left', on=['row_r'])

tmp['dist'] = np.hypot(tmp['LAT_h'].sub(tmp['LAT_r']), tmp['LON_h'].sub(tmp['LON_r']))

pysqldf = lambda q: sqldf(q, globals())
tmp = pysqldf("select HAEGU_NUM, LAT_h, LON_h, LAT_r, LON_r, min(dist) as dist from tmp group by row_r;")
redtide = pd.merge(redtide, tmp, how='left', on=['LAT_r','LON_r'])
              
redtide.info()
redtide.head(5)

#=============================================================
# HAEGU, period 선택
#=============================================================
#start = datetime.datetime.strptime("2008-09-19", "%Y-%m-%d").date()
#end = datetime.datetime.strptime("2016-12-31", "%Y-%m-%d").date()
start = np.datetime64("2008-09-19")
end = np.datetime64("2016-12-31")

redtide = redtide.loc[pd.to_datetime(redtide['YYMMDD']) >= start]
redtide = redtide.loc[pd.to_datetime(redtide['YYMMDD']) <= end]

redtide = redtide[redtide['month'].isin([5, 6, 7, 8, 9, 10, 11, 12])]
redtide = redtide[redtide['HAEGU_NUM'].isin([97, 98, 99, 213, 214])]

# 해구Num, 같은 날짜 여러 상태의 경우 Cochlo_YN값 MAX로 표출
redtide = redtide.groupby(['HAEGU_NUM', 'YYMMDD']).max()['Cochlo_YN'].reset_index()
redtide.head(5)

In [ ]:
#=============================================================
# 기상 데이터, 적조 데이터 결합
#=============================================================
AB = pd.merge(DT, redtide, how='left', on=['YYMMDD', 'HAEGU_NUM'])
AB['Cochlo_YN'] = AB['Cochlo_YN'].fillna(0)

# 데이터 결측치 확인
AB.isnull().sum()

In [ ]:
#=============================================================
# 데이터 전처리
#=============================================================
AB.describe()

# 기상변수에서 NA값이 -999로 처리된 경우 확인
AB[AB==-999]=np.nan

# 결측치 처리
AB = AB.fillna(AB.mean(numeric_only=True))

# 결측치 확인
AB.isnull().sum()

AB = AB.sort_values(by=['HAEGU_NUM', 'year', 'YYMMDD'], axis=0)

In [ ]:
#=============================================================
# 파생변수 생성
#=============================================================
# 7일 평균값을 구함
AB['mean_AVG_EMP'] = AB.groupby(['HAEGU_NUM', 'year'])['AVG_EMP'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MAX_SSH'] = AB.groupby(['HAEGU_NUM', 'year'])['MAX_SSH'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_AVG_SURFACE_SALINITY_TREND'] = AB.groupby(['HAEGU_NUM', 'year'])['AVG_SURFACE_SALINITY_TREND'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_STDEV_SURFACE_TEMPERATURE_TREND'] = AB.groupby(['HAEGU_NUM', 'year'])['STDEV_SURFACE_TEMPERATURE_TREND'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_AVG_SALINITY_01'] = AB.groupby(['HAEGU_NUM', 'year'])['AVG_SALINITY_01'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_AVG_SALINITY_02'] = AB.groupby(['HAEGU_NUM', 'year'])['AVG_SALINITY_02'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_AVG_SALINITY_03'] = AB.groupby(['HAEGU_NUM', 'year'])['AVG_SALINITY_03'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_AVG_SALINITY_04'] = AB.groupby(['HAEGU_NUM', 'year'])['AVG_SALINITY_04'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MAX_SALINITY_01'] = AB.groupby(['HAEGU_NUM', 'year'])['MAX_SALINITY_01'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MAX_SALINITY_02'] = AB.groupby(['HAEGU_NUM', 'year'])['MAX_SALINITY_02'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MAX_SALINITY_03'] = AB.groupby(['HAEGU_NUM', 'year'])['MAX_SALINITY_03'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MAX_SALINITY_04'] = AB.groupby(['HAEGU_NUM', 'year'])['MAX_SALINITY_04'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MIN_SALINITY_01'] = AB.groupby(['HAEGU_NUM', 'year'])['MIN_SALINITY_01'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MIN_SALINITY_02'] = AB.groupby(['HAEGU_NUM', 'year'])['MIN_SALINITY_02'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MIN_SALINITY_03'] = AB.groupby(['HAEGU_NUM', 'year'])['MIN_SALINITY_03'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MIN_SALINITY_04'] = AB.groupby(['HAEGU_NUM', 'year'])['MIN_SALINITY_04'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_STDEV_SALINITY_01'] = AB.groupby(['HAEGU_NUM', 'year'])['STDEV_SALINITY_01'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_STDEV_SALINITY_02'] = AB.groupby(['HAEGU_NUM', 'year'])['STDEV_SALINITY_02'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_STDEV_SALINITY_03'] = AB.groupby(['HAEGU_NUM', 'year'])['STDEV_SALINITY_03'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_STDEV_SALINITY_04'] = AB.groupby(['HAEGU_NUM', 'year'])['STDEV_SALINITY_04'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_AVG_TEMP_01'] = AB.groupby(['HAEGU_NUM', 'year'])['AVG_TEMP_01'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_AVG_TEMP_02'] = AB.groupby(['HAEGU_NUM', 'year'])['AVG_TEMP_02'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_AVG_TEMP_03'] = AB.groupby(['HAEGU_NUM', 'year'])['AVG_TEMP_03'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_AVG_TEMP_04'] = AB.groupby(['HAEGU_NUM', 'year'])['AVG_TEMP_04'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MAX_TEMP_01'] = AB.groupby(['HAEGU_NUM', 'year'])['MAX_TEMP_01'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MAX_TEMP_02'] = AB.groupby(['HAEGU_NUM', 'year'])['MAX_TEMP_02'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MAX_TEMP_03'] = AB.groupby(['HAEGU_NUM', 'year'])['MAX_TEMP_03'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MAX_TEMP_04'] = AB.groupby(['HAEGU_NUM', 'year'])['MAX_TEMP_04'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MIN_TEMP_01'] = AB.groupby(['HAEGU_NUM', 'year'])['MIN_TEMP_01'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MIN_TEMP_02'] = AB.groupby(['HAEGU_NUM', 'year'])['MIN_TEMP_02'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MIN_TEMP_03'] = AB.groupby(['HAEGU_NUM', 'year'])['MIN_TEMP_03'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MIN_TEMP_04'] = AB.groupby(['HAEGU_NUM', 'year'])['MIN_TEMP_04'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_STDEV_TEMP_01'] = AB.groupby(['HAEGU_NUM', 'year'])['STDEV_TEMP_01'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_STDEV_TEMP_02'] = AB.groupby(['HAEGU_NUM', 'year'])['STDEV_TEMP_02'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_STDEV_TEMP_03'] = AB.groupby(['HAEGU_NUM', 'year'])['STDEV_TEMP_03'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_STDEV_TEMP_04'] = AB.groupby(['HAEGU_NUM', 'year'])['STDEV_TEMP_04'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_AVG_U_VELOCITY_01'] = AB.groupby(['HAEGU_NUM', 'year'])['AVG_U_VELOCITY_01'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_AVG_U_VELOCITY_02'] = AB.groupby(['HAEGU_NUM', 'year'])['AVG_U_VELOCITY_02'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_AVG_U_VELOCITY_03'] = AB.groupby(['HAEGU_NUM', 'year'])['AVG_U_VELOCITY_03'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_AVG_U_VELOCITY_04'] = AB.groupby(['HAEGU_NUM', 'year'])['AVG_U_VELOCITY_04'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MAX_U_VELOCITY_01'] = AB.groupby(['HAEGU_NUM', 'year'])['MAX_U_VELOCITY_01'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MAX_U_VELOCITY_02'] = AB.groupby(['HAEGU_NUM', 'year'])['MAX_U_VELOCITY_02'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MAX_U_VELOCITY_03'] = AB.groupby(['HAEGU_NUM', 'year'])['MAX_U_VELOCITY_03'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MAX_U_VELOCITY_04'] = AB.groupby(['HAEGU_NUM', 'year'])['MAX_U_VELOCITY_04'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MIN_U_VELOCITY_01'] = AB.groupby(['HAEGU_NUM', 'year'])['MIN_U_VELOCITY_01'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MIN_U_VELOCITY_02'] = AB.groupby(['HAEGU_NUM', 'year'])['MIN_U_VELOCITY_02'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MIN_U_VELOCITY_03'] = AB.groupby(['HAEGU_NUM', 'year'])['MIN_U_VELOCITY_03'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MIN_U_VELOCITY_04'] = AB.groupby(['HAEGU_NUM', 'year'])['MIN_U_VELOCITY_04'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_STDEV_U_VELOCITY_01'] = AB.groupby(['HAEGU_NUM', 'year'])['STDEV_U_VELOCITY_01'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_STDEV_U_VELOCITY_02'] = AB.groupby(['HAEGU_NUM', 'year'])['STDEV_U_VELOCITY_02'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_STDEV_U_VELOCITY_03'] = AB.groupby(['HAEGU_NUM', 'year'])['STDEV_U_VELOCITY_03'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_STDEV_U_VELOCITY_04'] = AB.groupby(['HAEGU_NUM', 'year'])['STDEV_U_VELOCITY_04'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_AVG_V_VELOCITY_01'] = AB.groupby(['HAEGU_NUM', 'year'])['AVG_V_VELOCITY_01'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_AVG_V_VELOCITY_02'] = AB.groupby(['HAEGU_NUM', 'year'])['AVG_V_VELOCITY_02'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_AVG_V_VELOCITY_03'] = AB.groupby(['HAEGU_NUM', 'year'])['AVG_V_VELOCITY_03'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_AVG_V_VELOCITY_04'] = AB.groupby(['HAEGU_NUM', 'year'])['AVG_V_VELOCITY_04'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MAX_V_VELOCITY_01'] = AB.groupby(['HAEGU_NUM', 'year'])['MAX_V_VELOCITY_01'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MAX_V_VELOCITY_02'] = AB.groupby(['HAEGU_NUM', 'year'])['MAX_V_VELOCITY_02'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MAX_V_VELOCITY_03'] = AB.groupby(['HAEGU_NUM', 'year'])['MAX_V_VELOCITY_03'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MAX_V_VELOCITY_04'] = AB.groupby(['HAEGU_NUM', 'year'])['MAX_V_VELOCITY_04'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MIN_V_VELOCITY_01'] = AB.groupby(['HAEGU_NUM', 'year'])['MIN_V_VELOCITY_01'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MIN_V_VELOCITY_02'] = AB.groupby(['HAEGU_NUM', 'year'])['MIN_V_VELOCITY_02'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MIN_V_VELOCITY_03'] = AB.groupby(['HAEGU_NUM', 'year'])['MIN_V_VELOCITY_03'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_MIN_V_VELOCITY_04'] = AB.groupby(['HAEGU_NUM', 'year'])['MIN_V_VELOCITY_04'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_STDEV_V_VELOCITY_01'] = AB.groupby(['HAEGU_NUM', 'year'])['STDEV_V_VELOCITY_01'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_STDEV_V_VELOCITY_02'] = AB.groupby(['HAEGU_NUM', 'year'])['STDEV_V_VELOCITY_02'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_STDEV_V_VELOCITY_03'] = AB.groupby(['HAEGU_NUM', 'year'])['STDEV_V_VELOCITY_03'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7
AB['mean_STDEV_V_VELOCITY_04'] = AB.groupby(['HAEGU_NUM', 'year'])['STDEV_V_VELOCITY_04'].apply(lambda x : x.rolling(7).sum().shift(1)) / 7

# 전처리 이전 변수 삭제
AB.drop(['AVG_EMP', 'MAX_SSH', 'AVG_SURFACE_SALINITY_TREND', 'STDEV_SURFACE_TEMPERATURE_TREND', 'AVG_SALINITY_01', 'AVG_SALINITY_02'
, 'AVG_SALINITY_03', 'AVG_SALINITY_04', 'MAX_SALINITY_01', 'MAX_SALINITY_02', 'MAX_SALINITY_03', 'MAX_SALINITY_04', 'MIN_SALINITY_01'
, 'MIN_SALINITY_02', 'MIN_SALINITY_03', 'MIN_SALINITY_04', 'STDEV_SALINITY_01', 'STDEV_SALINITY_02', 'STDEV_SALINITY_03'
, 'STDEV_SALINITY_04', 'AVG_TEMP_01', 'AVG_TEMP_02', 'AVG_TEMP_03', 'AVG_TEMP_04', 'MAX_TEMP_01', 'MAX_TEMP_02', 'MAX_TEMP_03'
, 'MAX_TEMP_04', 'MIN_TEMP_01', 'MIN_TEMP_02', 'MIN_TEMP_03', 'MIN_TEMP_04', 'STDEV_TEMP_01', 'STDEV_TEMP_02', 'STDEV_TEMP_03'
, 'STDEV_TEMP_04', 'AVG_U_VELOCITY_01', 'AVG_U_VELOCITY_02', 'AVG_U_VELOCITY_03', 'AVG_U_VELOCITY_04', 'MAX_U_VELOCITY_01'
, 'MAX_U_VELOCITY_02', 'MAX_U_VELOCITY_03', 'MAX_U_VELOCITY_04', 'MIN_U_VELOCITY_01', 'MIN_U_VELOCITY_02', 'MIN_U_VELOCITY_03'
, 'MIN_U_VELOCITY_04', 'STDEV_U_VELOCITY_01', 'STDEV_U_VELOCITY_02', 'STDEV_U_VELOCITY_03', 'STDEV_U_VELOCITY_04'
, 'AVG_V_VELOCITY_01', 'AVG_V_VELOCITY_02', 'AVG_V_VELOCITY_03', 'AVG_V_VELOCITY_04', 'MAX_V_VELOCITY_01', 'MAX_V_VELOCITY_02'
, 'MAX_V_VELOCITY_03', 'MAX_V_VELOCITY_04', 'MIN_V_VELOCITY_01', 'MIN_V_VELOCITY_02', 'MIN_V_VELOCITY_03', 'MIN_V_VELOCITY_04'
, 'STDEV_V_VELOCITY_01', 'STDEV_V_VELOCITY_02', 'STDEV_V_VELOCITY_03', 'STDEV_V_VELOCITY_04'], axis='columns', inplace=True)


In [ ]:
# 14일 평균값을 구함
AB['mean_TA_MAX'] = AB.groupby(['HAEGU_NUM', 'year'])['TA_MAX'].apply(lambda x : x.rolling(14).sum().shift(1)) / 14
AB['mean_WS_MAX_ASOS'] = AB.groupby(['HAEGU_NUM', 'year'])['WS_MAX_ASOS'].apply(lambda x : x.rolling(14).sum().shift(1)) / 14
AB['mean_WS_MAX_BD'] = AB.groupby(['HAEGU_NUM', 'year'])['WS_MAX_BD'].apply(lambda x : x.rolling(14).sum().shift(1)) / 14
AB['mean_WS_INS'] = AB.groupby(['HAEGU_NUM', 'year'])['WS_INS'].apply(lambda x : x.rolling(14).sum().shift(1)) / 14
AB['mean_WS_MIN'] = AB.groupby(['HAEGU_NUM', 'year'])['WS_MIN'].apply(lambda x : x.rolling(14).sum().shift(1)) / 14
AB['mean_HM_AVG'] = AB.groupby(['HAEGU_NUM', 'year'])['HM_AVG'].apply(lambda x : x.rolling(14).sum().shift(1)) / 14
AB['mean_EV_L'] = AB.groupby(['HAEGU_NUM', 'year'])['EV_L'].apply(lambda x : x.rolling(14).sum().shift(1)) / 14
AB['mean_PA_AVG'] = AB.groupby(['HAEGU_NUM', 'year'])['PA_AVG'].apply(lambda x : x.rolling(14).sum().shift(1)) / 14
AB['mean_PS_MIN'] = AB.groupby(['HAEGU_NUM', 'year'])['PS_MIN'].apply(lambda x : x.rolling(14).sum().shift(1)) / 14
AB['mean_WH_MAX'] = AB.groupby(['HAEGU_NUM', 'year'])['WH_MAX'].apply(lambda x : x.rolling(14).sum().shift(1)) / 14

# 전처리 이전 변수 삭제
AB.drop(['TA_MAX', 'WS_MAX_ASOS', 'WS_MAX_BD', 'WS_INS', 'WS_MIN', 'HM_AVG', 'EV_L', 'PA_AVG', 'PS_MIN', 'WH_MAX'], axis='columns', inplace=True)

# 14일 누적치를 구함
AB['sum_SS_DAY'] = AB.groupby(['HAEGU_NUM', 'year'])['SS_DAY'].apply(lambda x : x.rolling(14).sum().shift(1))
AB['sum_RN_DAY'] = AB.groupby(['HAEGU_NUM', 'year'])['RN_DAY'].apply(lambda x : x.rolling(14).sum().shift(1))
AB['sum_SI_DAY'] = AB.groupby(['HAEGU_NUM', 'year'])['SI_DAY'].apply(lambda x : x.rolling(14).sum().shift(1))
AB['sum_RN_DUR'] = AB.groupby(['HAEGU_NUM', 'year'])['RN_DUR'].apply(lambda x : x.rolling(14).sum().shift(1))
AB.drop(['SS_DAY', 'RN_DAY', 'SI_DAY', 'RN_DUR'], axis='columns', inplace=True)


In [ ]:
# time interval create
AB['Cochlo_YN'] = AB.groupby(['HAEGU_NUM', 'year'])['Cochlo_YN'].shift(-7)

#start = datetime.datetime.strptime("2008-10-04", "%Y-%m-%d").date()
#end = datetime.datetime.strptime("2016-12-31", "%Y-%m-%d").date()
start = np.datetime64("2008-10-04")
end = np.datetime64("2016-12-31")

AB = AB.loc[pd.to_datetime(AB['YYMMDD']) >= start]
AB = AB.loc[pd.to_datetime(AB['YYMMDD']) <= end]

AB = AB[AB['month'].isin([5, 6, 7, 8, 9, 10, 11])]

#======================================================================================================
#메모리 용량 줄이기
#======================================================================================================
del(start, end)
del(ASOS,BUOY_DP,DT,HAEGU,HYCOM,redtide,tmp,match)

AB.info()
AB.head(5)

In [ ]:
#=============================================================
# 분석
#=============================================================
h2o.init(max_mem_size = "4G", nthreads = 1)
h2o.remove_all()

In [ ]:
# setting AB data
dataXY = AB
dataXY = dataXY.drop(['HAEGU_NUM', 'YYMMDD', 'month', 'year'], axis=1)
dataXY = dataXY.rename(columns = {"Cochlo_YN":"Y"})
dataXY.reset_index(drop=True, inplace=True)
dataXY.info()

# train, valid, test 데이터 분리
## split to train, valid, test 
dataXY = h2o.H2OFrame(dataXY)
train_data, valid_data, test_data = dataXY.split_frame(ratios=[0.7,0.15], seed=1111)

## 독립변수, 종속변수 설정(x: 독립변수, y: 종속변수)
x = dataXY.columns
x.remove('Y')
y='Y'

train_data['Y'] = train_data['Y'].asfactor()
valid_data['Y'] = valid_data['Y'].asfactor()
test_data['Y'] = test_data['Y'].asfactor()

In [ ]:
# -----------------------------------------------------------------------------
# 모형 튜닝 자동화
# -----------------------------------------------------------------------------
# cartesian grid search 
# -----------------------------------------------------------------------------
hyper_parameters = {'max_depth': [4, 6, 8, 12, 16, 20]}

# 조합 모형 돌리기
m = H2OGridSearch(H2ORandomForestEstimator,
                  hyper_params=hyper_parameters,
                  search_criteria={'strategy': "Cartesian"},
                  grid_id='RF_depth_grid')

m.train(x = x,
        y = y,
        training_frame = train_data,
        validation_frame = valid_data,
        ntrees = 10000,
        stopping_rounds = 5,
        stopping_tolerance = 1e-4,
        stopping_metric = 'AUC',
        score_tree_interval = 5,
        seed=1111)

# AUC가 높은 순으로 정렬하기
sortedGrid = m.get_grid(sort_by='auc', decreasing=True)

print('===== sortedGrid =====')
print(sortedGrid)

In [ ]:
## 모형 튜닝 자동화
minDepth = 12
maxDepth = 16

# options for grid search 
max_runtime_secs = 60*10
max_models = 100

# random grid search 
hyper_params = {
    'max_depth': list(range(minDepth, maxDepth + 1)),
    'sample_rate': [i * 0.01 for i in range(20, 100 + 1)],
    'col_sample_rate_per_tree': [i * 0.01 for i in range(20, 100 + 1)],
    'col_sample_rate_change_per_level': [i * 0.01 for i in range(90, 110 + 1)],
    'min_rows': [1,5,10,20,50,100],
    'min_split_improvement': [0,1e-8,1e-6,1e-4],
    'histogram_type': ['UniformAdaptive', 'QuantilesGlobal', 'RoundRobin']
}

search_criteria = {
    'strategy': "RandomDiscrete",
    'max_runtime_secs': max_runtime_secs,
    'max_models': max_models
}

grid = H2OGridSearch(H2ORandomForestEstimator
                     , hyper_params=hyper_parameters
                     , search_criteria=search_criteria
                     , grid_id='RF_grid')
grid.train(
      x = x
    , y = y
    , training_frame = train_data
    , validation_frame = valid_data
    , ntrees = 10000
    , stopping_rounds = 5
    , stopping_tolerance = 1e-4
    , stopping_metric = 'AUC'
    , score_tree_interval = 5
    , seed = 1111
)

# AUC가 높은 순으로 정렬하기
sortedGrid = grid.get_grid(sort_by='auc', decreasing=True)
RF_AB_Tune = h2o.get_model(sortedGrid.model_ids[1])
print(RF_AB_Tune)

In [ ]:
#======================================================================================================
# forecast
#======================================================================================================
pred= RF_AB_Tune.predict(test_data)
pred=pred.as_data_frame()
test_data=test_data.as_data_frame()

pred = pd.concat([pred['predict'], pred['p1'], test_data['Y']], axis=1)
pred.columns = ['Yhat','p1','Y']

#confusion matrix
print(confusion_matrix(pred['Yhat'],pred['Y']),
      classification_report(pred['Yhat'],pred['Y']))